Closely follows Andrej's Karpathy's [Let's build GPT](https://youtu.be/kCc8FmEb1nY). 

In [1]:
import torch 
import torch.nn as nn
from torch.nn import functional as F 
torch.manual_seed(1337)

from functools import partial, reduce 

In [2]:
with open("data/shakespeare.txt") as f: 
    text = f.read()

print(text[200:500])

 know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(' '.join(chars))
print(f"vocab size: {vocab_size}")


   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z
vocab size: 65


Defines tokenizer encoder and decoder:

In [4]:
int_of_char = {ch:i for i, ch in enumerate(chars)}
char_of_int = {i:ch for i, ch in enumerate(chars)}

encode = lambda text: [int_of_char[ch] for ch in text]
decode = lambda toks: reduce(lambda x, y: x + y, 
                             [char_of_int[i] for i in toks]
                            )

print(encode("hi there!"))
print(decode(encode("hi there!")))

[46, 47, 1, 58, 46, 43, 56, 43, 2]
hi there!


Let's wrap the entire dataset in a tensor

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)

print(data[:100])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


Train/val split. 

In [6]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

We use context lengths of 8. 

In [7]:
context_length = 8
batch_size = 4

train_data[:context_length+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [8]:
x = train_data[:context_length]
y = train_data[1:context_length+1]
for t in range(context_length):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} target is {target}")
    

when input is tensor([18]) target is 47
when input is tensor([18, 47]) target is 56
when input is tensor([18, 47, 56]) target is 57
when input is tensor([18, 47, 56, 57]) target is 58
when input is tensor([18, 47, 56, 57, 58]) target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


We implement batching, i.e stacking multiple sequences

In [9]:
def get_random_batch(data, context_length=8, batch_size=4): 
    idxs = torch.randint(len(data)-context_length, (batch_size,))
    x = torch.stack([data[i:i+context_length] for i in idxs])
    y = torch.stack([data[i+1:i+context_length+1] for i in idxs])
    return x, y

In [10]:
get_random_batch(train_data)

(tensor([[24, 43, 58,  5, 57,  1, 46, 43],
         [44, 53, 56,  1, 58, 46, 39, 58],
         [52, 58,  1, 58, 46, 39, 58,  1],
         [25, 17, 27, 10,  0, 21,  1, 54]]),
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
         [53, 56,  1, 58, 46, 39, 58,  1],
         [58,  1, 58, 46, 39, 58,  1, 46],
         [17, 27, 10,  0, 21,  1, 54, 39]]))

The mathematical trick in self-attention

In [23]:
B, T, C = 4, 8, 2 # batch, time, channel
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

Suppose we want the representation of a token to be the *average* of all the tokens that came before it. We can do this with a matmul! 

In [48]:
lowert = torch.tril(torch.ones(T, T))
avg = lowert/torch.sum(lowert, 1, keepdim=True)
avg

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [79]:
# another way of making the same matrix. This is the way that generalizes to self-attention. 
# can think of matrix entries as affinities or interaction strengths
# these affinities are going to become data dependent. 
wei = torch.ones((T, T))
wei = wei.masked_fill(torch.tril(wei)==0, float('-inf'))
avg = F.softmax(wei, dim=1)
avg

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [80]:
print(f"old representations (batch element 0):\n {x[0]}")
print(f"new representations (batch element 0):\n {(avg @ x)[0]}")

old representations (batch element 0):
 tensor([[-0.4970,  0.4658],
        [-0.2573, -1.0673],
        [ 0.8353, -1.9560],
        [-0.8003, -0.5045],
        [-1.4267,  0.9059],
        [ 0.1446,  0.2280],
        [-0.2282, -0.6885],
        [ 0.1832,  0.6004]])
new representations (batch element 0):
 tensor([[-0.4970,  0.4658],
        [-0.3771, -0.3008],
        [ 0.0270, -0.8525],
        [-0.1798, -0.7655],
        [-0.4292, -0.4312],
        [-0.3335, -0.3214],
        [-0.3185, -0.3738],
        [-0.2558, -0.2520]])


Now we implement self-attention, where the affinities are learned. 

In [97]:
# single self-attention head 
tril = torch.tril(torch.ones(T, T))
head_size = 16 
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)
v= value(x) # (B, T, C)
wei = q @ k.transpose(-2, -1) * head_size**-.5

wei = wei.masked_fill(tril==0, float('-inf')) # delete to get encoder
wei = F.softmax(wei, dim=-1)

out= wei @ v 
print(x.shape)
print(wei[0])
print(out.shape)

torch.Size([4, 8, 2])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4982, 0.5018, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3911, 0.3017, 0.3072, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2242, 0.2592, 0.2620, 0.2546, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1601, 0.2244, 0.2238, 0.2179, 0.1738, 0.0000, 0.0000, 0.0000],
        [0.1683, 0.1650, 0.1644, 0.1655, 0.1681, 0.1687, 0.0000, 0.0000],
        [0.1415, 0.1440, 0.1454, 0.1431, 0.1409, 0.1418, 0.1434, 0.0000],
        [0.1257, 0.1242, 0.1232, 0.1248, 0.1263, 0.1254, 0.1246, 0.1258]],
       grad_fn=<SelectBackward0>)
torch.Size([4, 8, 16])


You can think about attention as communication in a weighted complete graph

This is self-attention. In cross-attention, the queries are produced from some separate source $y$, and the keys and values are produced from $x$. 

Want to do all you can to prevent attention scores from degenerating to one-hot vectors. 